In [41]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from matplotlib import animation
from skimage import io
import os
from IPython.display import HTML
import numpy as np

In [2]:
path_img = '/home/mougeotg/all/data/nuclei/gred_val_all/Task500_Nucleus/imagesTs/Nucleus_009.nii.gz'
folder_in = '/home/mougeotg/all/data/nuclei/gred_val_all/Task500_Nucleus/imagesTs/'
folder_out = '/home/mougeotg/all/data/nuclei/gred_val_all/masks_nnunet/images/'

In [3]:
# list_img = os.listdir(folder_in)
# for i in range(len(list_img)):
#     end = list_img[i][list_img[i].rfind('.'):]
#     begin = list_img[i][:list_img[i].rfind('.')]
#     begin = begin[:begin.rfind('.')]
#     if end=='.gz':
#         img = sitk.ReadImage(folder_in + list_img[i])
#         img_np = sitk.GetArrayFromImage(img)
        
#         # rerange the pixel
#         # img_np = 255*img_np
        
        
#         io.imsave(folder_out + begin + '.tif', img_np)

In [4]:
def nii2np_single(img_path):
    """
    convert nifti format (.nii.gz) to numpy array
    """
    img = sitk.ReadImage(img_path)
    img_np = sitk.GetArrayFromImage(img)
    return img_np

In [5]:
def nii2tif_single(nii_path, tif_path, resample):
    """
    load a nifti file and save it into a tif
    """
    img = nii2np_single(nii_path)
    img = resample(img)
    io.imsave(tif_path, img)

In [6]:
def abs_path(root, listdir_):
    """
    absolute path
    add root to the beginning of each path in listdir
    """
    listdir = listdir_.copy()
    for i in range(len(listdir)):
        listdir[i] = root + '/' + listdir[i]
    return listdir

def abs_listdir(path):
    """
    absolute path
    read all the path of files stored in 'path' 
    and add root to the beginning of each path in listdir
    """
    return abs_path(path, os.listdir(path))

In [7]:
def nii2tif_folder(nii_folder, tif_folder, resample):
    """
    load a folder of nifti file and save it into a folder of tif
    """
    list_rel = os.listdir(nii_folder)
    list_abs = abs_listdir(nii_folder)
    for i, nii_path in enumerate(list_abs):
        print('Loading index: {:d}/{}'.format(i+1, len(list_abs)), end='')
        print('{:s}\r'.format(''), end='', flush=True)
        
        end = list_rel[i][list_rel[i].rfind('.'):]
        if end=='.gz': # assert it is a nifti file
            tif_path = list_rel[i][:list_rel[i].rfind('.')]
            tif_path = tif_path[:tif_path.rfind('.')]
            tif_path = os.path.join(tif_folder, tif_path+'.tif')
            nii2tif_single(nii_path, tif_path, resample)

### application to self-supervised-3d pancreas segmentation from medical segmentation decathlon

In [47]:
# img_path = "/home/gumougeot/all/data/Task07_Pancreas/imagesTr/pancreas_001.nii.gz"
img_path = "/home/gumougeot/all/codes/python/3dnucleus/data/nucleus/images_resampled_sophie/KAKU4-wt--CRWN1-wt--CRWN4-wt_Cot_J13_STD_FIXE_H258_X3_7_C0.tif"
# img_path = "/home/gumougeot/all/data/Task07_Pancreas/labelsTr/pancreas_001.nii.gz"

nii_folder = "/home/gumougeot/all/data/Task07_Pancreas/imagesTr/"
tif_folder = "/home/gumougeot/all/data/Task07_Pancreas/tif_imagesTr/"

# nii_folder = "/home/gumougeot/all/data/Task07_Pancreas/labelsTr/"
# tif_folder = "/home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/"

import torchio as tio
def resample(img, size=(128,128,128), rerange_image=False, rerange_label=False):
    transform = tio.transforms.Resize(target_shape=size)
    img_tmp = np.expand_dims(img,0)
    # for label: rerange 
#     print(np.max(img_tmp))
    img_tmp = transform(img_tmp)
    if rerange_label:
#         print(type(img_tmp[0][0][0][0]))
        img_tmp = (img_tmp > 0).astype(np.uint8)*255
        img_tmp = img_tmp.astype(np.uint8)
        if len(np.unique(img_tmp[0]))!=2:
            print('error')
    elif rerange_image:
        img_tmp = (img_tmp - img_tmp.min()) / (img_tmp.max() - img_tmp.min())
    return img_tmp[0]

# test
img = nii2np_single(img_path)
# img_ = resample(img, rerange=True)
print(np.unique(img))
print(img.shape)
img_ = resample(img, rerange_image=True)
print(np.unique(img_))
print(img_.shape)

[    0    12    14 ...  9716  9909 10140]
(128, 128, 128)
[0.         0.00118343 0.00138067 ... 0.9581854  0.97721893 1.        ]
(128, 128, 128)


In [13]:
nii2tif_folder(nii_folder, tif_folder, resample=lambda x: resample(x, rerange=True))

/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_041.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_130.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_091.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_293.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_369.tif is a low contrast image
  io.imsave(tif_path, img)


...

/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_119.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_269.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_296.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_387.tif is a low contrast image
  io.imsave(tif_path, img)


/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_246.tif is a low contrast image
  io.imsave(tif_path, img)
/tmp/ipykernel_20848/1548559800.py:7: UserWarning: /home/gumougeot/all/data/Task07_Pancreas/tif_labelsTr/pancreas_183.tif is a low contrast image
  io.imsave(tif_path, img)


In [15]:
def get_js_video(images, batch=0, channel=0, cmap='jet',
                 vmin=0, vmax=0, interval=200, repeat_delay=1000):
    """Create a JavaScript video as HTML for visualizing 3D data as a movie.

    Args:
        images (numpy.array): images to display as video
        batch (int): batch number of images to plot
        channel (int): channel index to plot
        vmin (int): lower end of data range covered by colormap
        vmax (int): upper end of data range covered by colormap

    Returns:
        str: JS HTML to display video
    """
    fig = plt.figure()

    ims = []
    plot_kwargs = {
        'animated': True,
        'cmap': cmap,
    }

    if vmax == 0:
        vmax = images.max()

    # TODO: do these not work for other cmaps?
    if cmap == 'cubehelix' or cmap == 'jet':
        plot_kwargs['vmin'] = vmin
        plot_kwargs['vmax'] = vmax

    for i in range(images.shape[1]):
        im = plt.imshow(images[batch, i, :, :, channel], **plot_kwargs)
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=interval, repeat_delay=repeat_delay)
    plt.close()
    return ani.to_jshtml()

In [17]:
img_display = np.expand_dims(img_, axis=(0,-1))
img_display = (img_display==1).astype(np.uint8)
HTML(get_js_video(img_display))